# Previsione del fallimento
In questo progetto è stato preso in esame un dataset, contenente dati riguardanti migliaia di aziende, al fine di sviluppare un modello capace di prevedere la possibilità che un'azienda possa o meno fallire.

In [47]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

# Import datasets

In [2]:
train = pd.read_csv('datitrain.csv')
train

,ID,Target,Z1,Z2,Regione,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,...,TOTALEPAS-1,TOTALEPAS-2,TOTALEDEBITI-1,TOTALEDEBITI-2,Totaldebitientro-1,Totaldebitientro-2,Totaldebitioltre-1,Totaldebitioltre-2,RedditivROI-1,RedditivROI-2
0,7358,0,1.479190,1.695080,VENETO,3618,3337,26,24,-371,...,3491,3577,1461,1137,1461,1137,0,0,-25.53,-7.45
1,9744,0,1.547276,1.721152,TRENTINO-ALTO,1735,1843,2,2,281,...,2244,2425,855,851,855,851,0,0,10.12,16.51
2,13087,0,1.102310,1.526099,CAMPANIA,1098,1166,10,13,93,...,2234,1493,1580,866,596,584,984,282,4.26,3.54
3,7510,0,3.824231,3.456398,SICILIA,9187,9081,44,45,362,...,3055,3336,1994,2428,332,1438,1662,990,10.68,12.78
4,8450,0,3.082321,2.647675,PUGLIA,2712,2426,14,13,113,...,1323,1513,1074,1171,963,1057,110,115,24.54,19.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,1241,0,2.864663,2.909667,LOMBARDIA,3955,3589,16,15,426,...,2600,2508,1345,1268,1176,1104,169,164,0.00,0.00
10996,11685,0,1.481994,1.435089,SICILIA,1152,1006,4,4,85,...,1839,1599,1588,1366,1070,942,518,424,5.42,4.95
10997,2348,0,3.785777,6.958232,LOMBARDIA,2035,2595,1,1,-9,...,659,435,547,296,547,296,0,0,-1.73,8.73
10998,14184,1,2.516040,1.096166,TOSCANA,13632,11437,123,127,418,...,6484,9590,4959,10716,4908,10540,51,176,9.86,0.00


In [3]:
test = pd.read_csv('datitest.csv')
test

,ID,Z1,Z2,Regione,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,...,TOTALEPAS-1,TOTALEPAS-2,TOTALEDEBITI-1,TOTALEDEBITI-2,Totaldebitientro-1,Totaldebitientro-2,Totaldebitioltre-1,Totaldebitioltre-2,RedditivROI-1,RedditivROI-2
0,1,1.118565,0.960366,PIEMONTE,1693,1531,13,11,308,217,...,2873,2821,1775,1701,947,954,827,747,6.35,1.08
1,2,1.925179,2.008993,EMILIA-ROMAGNA,3533,3795,14,14,926,873,...,5611,5027,2499,2389,797,802,1702,1587,13.01,14.30
2,3,1.519329,1.387092,CALABRIA,2487,2537,37,43,702,423,...,4951,4849,3252,3407,2182,2396,1070,1011,14.47,4.17
3,4,1.639071,1.548504,CAMPANIA,1556,1570,20,20,133,127,...,2330,2525,1572,1847,1478,1739,94,108,19.70,19.75
4,5,1.615415,2.028106,LOMBARDIA,7260,6550,20,20,640,622,...,8449,5445,7042,4613,5481,3555,1561,1058,6.11,14.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,3962,4.137058,3.743558,LAZIO,2961,2263,32,23,524,239,...,1348,985,804,741,804,725,0,16,0.00,0.00
3962,3963,3.489946,3.571986,PIEMONTE,750,694,6,7,11,35,...,263,262,156,163,155,162,1,1,7.51,0.00
3963,3964,1.611348,2.366697,TOSCANA,1036,1785,0,0,159,369,...,1595,1677,622,780,356,454,266,327,8.57,24.40
3964,3965,1.238945,1.066325,PUGLIA,1251,1101,8,8,23,-78,...,2845,2862,867,773,439,773,428,0,-2.25,-6.89


# Pulizia Train

In questa fase eseguiremo la pulizia dei dati del dataset di Train

In [4]:
train = train.interpolate()

In [5]:
train.describe()

,ID,Target,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,...,TOTALEPAS-1,TOTALEPAS-2,TOTALEDEBITI-1,TOTALEDEBITI-2,Totaldebitientro-1,Totaldebitientro-2,Totaldebitioltre-1,Totaldebitioltre-2,RedditivROI-1,RedditivROI-2
count,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,...,1.100000e+04,1.100000e+04,1.100000e+04,1.100000e+04,11000.000000,11000.000000,1.100000e+04,1.100000e+04,11000.000000,11000.000000
mean,7486.053455,0.075636,2.144992,2.151810,3804.083545,3782.112000,19.549182,18.989636,235.321727,246.023727,...,6.278804e+03,6.289652e+03,4.378813e+03,4.292990e+03,3015.587091,2963.186727,1.363229e+03,1.329807e+03,6.943755,6.750372
std,4326.628928,0.264427,1.316907,1.360106,10013.028610,10235.415848,53.812624,52.708710,2196.847795,1873.507900,...,3.914484e+04,3.692163e+04,2.351706e+04,2.288189e+04,15161.592883,16007.219703,1.459139e+04,1.329493e+04,9.302519,9.425980
min,1.000000,0.000000,-9.534840,-8.934343,4.000000,2.000000,0.000000,0.000000,-76863.000000,-48815.000000,...,8.000000e+01,3.000000e+01,4.400000e+01,8.000000e+00,7.000000,8.000000,0.000000e+00,-5.000000e+00,-29.990000,-29.980000
25%,3728.750000,0.000000,1.454048,1.438546,1070.000000,1014.750000,6.000000,5.000000,56.000000,51.000000,...,9.600000e+02,8.900000e+02,6.190000e+02,5.890000e+02,472.000000,455.000000,1.700000e+01,7.000000e+00,0.000000,0.000000
50%,7485.500000,0.000000,2.035230,2.016055,1802.000000,1733.000000,11.000000,10.000000,122.000000,115.000000,...,1.799000e+03,1.715500e+03,1.192000e+03,1.133000e+03,895.000000,868.000000,1.480000e+02,1.220000e+02,5.940000,5.810000
75%,11233.250000,0.000000,2.659017,2.653851,3427.500000,3307.000000,18.000000,17.000000,261.000000,248.000000,...,3.767000e+03,3.615250e+03,2.482250e+03,2.407250e+03,1792.250000,1762.250000,5.310000e+02,4.720000e+02,12.570000,12.350000
max,14966.000000,1.000000,29.369867,44.735907,189321.000000,199019.000000,2449.000000,2571.000000,82087.000000,74881.000000,...,2.769359e+06,2.371103e+06,1.233604e+06,1.254761e+06,519404.000000,850002.000000,1.147745e+06,1.125097e+06,29.980000,29.980000


Rimuoviamo ID e le variabili Regione e Target per eseguire la normalizzazione dei dati.

In [6]:
train = train.loc[:, train.columns != 'ID']
train_cleaned = train.loc[:, train.columns != "Regione"]
train_cleaned = train_cleaned.loc[:, train_cleaned.columns != "Target"]

In [7]:
train_normalized = preprocessing.normalize(train_cleaned, norm = "l2", axis = 0, copy = True, return_norm = False)

In [8]:
# Trasformiamo train_norm in un dataframe
train_normalized = pd.DataFrame(train_normalized, columns = train_cleaned.columns)

In [9]:
# Riassegnare il nome delle colonne
train_normalized["Regione"] = train["Regione"]
train_normalized["Target"] = train["Target"]

In [10]:
# Verifichiamo il numero dei valori della variabile Regione
len(train_normalized["Regione"].value_counts())

24

In [11]:
train_normalized["Regione"].value_counts()

LOMBARDIA              2080
VENETO                 1334
LAZIO                  1014
EMILIA-ROMAGNA          979
PIEMONTE                866
CAMPANIA                835
TOSCANA                 830
SICILIA                 682
PUGLIA                  550
MARCHE                  307
LIGURIA                 271
FRIULI-VENEZIA          241
TRENTINO-ALTO           211
ABRUZZO                 197
SARDEGNA                167
CALABRIA                151
UMBRIA                  150
BASILICATA               61
MOLISE                   33
TRENTINOALTOADIGE        17
VALLED'AOSTA/            14
FRIULIVENEZIAGIULIA       6
EMILIAROMAGNA             2
VALLED'AOSTA              2
Name: Regione, dtype: int64

Nella variabile Regione sono presente dei valori diversi che fanno riferimento alla stessa regione, per questo motivo procediamo a unificarli.

In [ ]:
for reg in range(len(train_normalized['Regione'])):
    if train_normalized['Regione'][reg][0:5] == 'VALLE':
        train_normalized['Regione'][reg] = 'VALLEDAOSTA'
    elif train_normalized['Regione'][reg][0:5] == 'TRENT':
        train_normalized['Regione'][reg] = 'TRENTINOALTOADIGE'
    elif train_normalized['Regione'][reg][0:5] == 'EMILI':
        train_normalized['Regione'][reg] = 'EMILIAROMAGNA'
    elif train_normalized['Regione'][reg][0:5] == 'FRIUL':
        train_normalized['Regione'][reg] = 'FRIULIVENEZIAGIULIA'
    elif train_normalized['Regione'][reg][0:5] == 'MATER':
        train_normalized['Regione'][reg] = 'BASILICATA'

In [13]:
train = pd.get_dummies(train_normalized, columns=['Regione'], prefix=['Regione'])

# Pulizia Test

In [14]:
# Controlliamo quante osservazioni abbiamo per regione, così come abbiamo fatto per il dataset di train
len(test["Regione"].value_counts())

24

In [15]:
test["Regione"].value_counts()

LOMBARDIA              743
VENETO                 477
LAZIO                  376
EMILIA-ROMAGNA         343
PIEMONTE               318
CAMPANIA               314
TOSCANA                294
SICILIA                225
PUGLIA                 199
MARCHE                 123
ABRUZZO                 93
LIGURIA                 91
FRIULI-VENEZIA          73
TRENTINO-ALTO           70
UMBRIA                  61
CALABRIA                60
SARDEGNA                47
BASILICATA              23
MOLISE                  17
VALLED'AOSTA/            8
TRENTINOALTOADIGE        6
FRIULIVENEZIAGIULIA      2
EMILIAROMAGNA            2
MATERA                   1
Name: Regione, dtype: int64

In [16]:
# Droppiamo le variabili ID e Regione per normalizzare
test_cleaned = test.loc[:, test.columns != 'ID']
test_cleaned = test_cleaned.loc[:, test_cleaned.columns != "Regione"]

In [17]:
test_normalized = preprocessing.normalize(test_cleaned, norm = "l2", axis = 0, copy = True, return_norm = False)

In [18]:
# Trasformiamo test_norm in un dataframe
test_normalized = pd.DataFrame(test_normalized, columns = test_cleaned.columns)

In [19]:
# Reinseriamo le regioni
test_normalized["Regione"] = test["Regione"]

In [20]:
test_normalized

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,TOTALEPAS-2,TOTALEDEBITI-1,TOTALEDEBITI-2,Totaldebitientro-1,Totaldebitientro-2,Totaldebitioltre-1,Totaldebitioltre-2,RedditivROI-1,RedditivROI-2,Regione
0,0.007146,0.006096,0.002404,0.002098,0.003685,0.003206,0.002577,0.002575,0.000622,0.000496,...,0.000496,0.000577,0.000409,0.000842,0.000630,3.302147e-04,2.626748e-04,0.008815,0.001485,PIEMONTE
1,0.012299,0.012753,0.005016,0.005201,0.003969,0.004081,0.007749,0.010358,0.001216,0.000884,...,0.000884,0.000813,0.000575,0.000709,0.000530,6.795955e-04,5.580521e-04,0.018060,0.019661,EMILIA-ROMAGNA
2,0.009706,0.008805,0.003531,0.003477,0.010489,0.012534,0.005874,0.005019,0.001073,0.000853,...,0.000853,0.001057,0.000820,0.001940,0.001583,4.272428e-04,3.555076e-04,0.020087,0.005733,CALABRIA
3,0.010471,0.009830,0.002209,0.002152,0.005670,0.005830,0.001113,0.001507,0.000505,0.000444,...,0.000444,0.000511,0.000444,0.001314,0.001149,3.753348e-05,3.797708e-05,0.027347,0.027154,CAMPANIA
4,0.010320,0.012874,0.010307,0.008977,0.005670,0.005830,0.005356,0.007380,0.001830,0.000958,...,0.000958,0.002290,0.001110,0.004873,0.002349,6.232953e-04,3.720347e-04,0.008482,0.020087,LOMBARDIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,0.026429,0.023763,0.004204,0.003101,0.009072,0.006704,0.004385,0.002836,0.000292,0.000173,...,0.000173,0.000261,0.000178,0.000715,0.000479,0.000000e+00,5.626234e-06,0.000000,0.000000,LAZIO
3962,0.022295,0.022674,0.001065,0.000951,0.001701,0.002040,0.000092,0.000415,0.000057,0.000046,...,0.000046,0.000051,0.000039,0.000138,0.000107,3.992923e-07,3.516396e-07,0.010425,0.000000,PIEMONTE
3963,0.010294,0.015023,0.001471,0.002446,0.000000,0.000000,0.001331,0.004378,0.000346,0.000295,...,0.000295,0.000202,0.000188,0.000317,0.000300,1.062118e-04,1.149862e-04,0.011897,0.033547,TOSCANA
3964,0.007915,0.006769,0.001776,0.001509,0.002268,0.002332,0.000192,-0.000925,0.000616,0.000503,...,0.000503,0.000282,0.000186,0.000390,0.000511,1.708971e-04,0.000000e+00,-0.003123,-0.009473,PUGLIA


In [ ]:
for element in range(len(test_normalized["Regione"])):   
  if test_normalized["Regione"][element][0:13] == "VALLED'AOSTA/":         
    test_normalized["Regione"][element] = "VALLEDAOSTA"     
  elif test_normalized["Regione"][element][0:13] == "TRENTINO-ALTO":         
    test_normalized["Regione"][element] = "TRENTINOALTOADIGE"     
  elif test_normalized["Regione"][element][0:14] == "EMILIA-ROMAGNA":         
    test_normalized["Regione"][element] = "EMILIAROMAGNA"     
  elif test_normalized["Regione"][element][0:14] == "FRIULI-VENEZIA":         
    test_normalized["Regione"][element] = "FRIULIVENEZIAGIULIA"     
  elif test_normalized["Regione"][element][0:6] == "MATERA":         
    test_normalized["Regione"][element] = "BASILICATA"

In [22]:
len(test_normalized["Regione"].value_counts())

20

In [23]:
test_normalized["Regione"].value_counts()

LOMBARDIA              743
VENETO                 477
LAZIO                  376
EMILIAROMAGNA          345
PIEMONTE               318
CAMPANIA               314
TOSCANA                294
SICILIA                225
PUGLIA                 199
MARCHE                 123
ABRUZZO                 93
LIGURIA                 91
TRENTINOALTOADIGE       76
FRIULIVENEZIAGIULIA     75
UMBRIA                  61
CALABRIA                60
SARDEGNA                47
BASILICATA              24
MOLISE                  17
VALLEDAOSTA              8
Name: Regione, dtype: int64

In [24]:
new_test = pd.get_dummies(test_normalized, columns=['Regione'], prefix=['Regione'])
new_test

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,Regione_MOLISE,Regione_PIEMONTE,Regione_PUGLIA,Regione_SARDEGNA,Regione_SICILIA,Regione_TOSCANA,Regione_TRENTINOALTOADIGE,Regione_UMBRIA,Regione_VALLEDAOSTA,Regione_VENETO
0,0.007146,0.006096,0.002404,0.002098,0.003685,0.003206,0.002577,0.002575,0.000622,0.000496,...,0,1,0,0,0,0,0,0,0,0
1,0.012299,0.012753,0.005016,0.005201,0.003969,0.004081,0.007749,0.010358,0.001216,0.000884,...,0,0,0,0,0,0,0,0,0,0
2,0.009706,0.008805,0.003531,0.003477,0.010489,0.012534,0.005874,0.005019,0.001073,0.000853,...,0,0,0,0,0,0,0,0,0,0
3,0.010471,0.009830,0.002209,0.002152,0.005670,0.005830,0.001113,0.001507,0.000505,0.000444,...,0,0,0,0,0,0,0,0,0,0
4,0.010320,0.012874,0.010307,0.008977,0.005670,0.005830,0.005356,0.007380,0.001830,0.000958,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,0.026429,0.023763,0.004204,0.003101,0.009072,0.006704,0.004385,0.002836,0.000292,0.000173,...,0,0,0,0,0,0,0,0,0,0
3962,0.022295,0.022674,0.001065,0.000951,0.001701,0.002040,0.000092,0.000415,0.000057,0.000046,...,0,1,0,0,0,0,0,0,0,0
3963,0.010294,0.015023,0.001471,0.002446,0.000000,0.000000,0.001331,0.004378,0.000346,0.000295,...,0,0,0,0,0,1,0,0,0,0
3964,0.007915,0.006769,0.001776,0.001509,0.002268,0.002332,0.000192,-0.000925,0.000616,0.000503,...,0,0,1,0,0,0,0,0,0,0


# Creazione Train e Validate

In [25]:
train_df, validation = train_test_split(train, test_size = 0.2, random_state = 45)

Ora sistemiamo lo sbilanciamento del dataset.

In [26]:
train_df

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,Regione_MOLISE,Regione_PIEMONTE,Regione_PUGLIA,Regione_SARDEGNA,Regione_SICILIA,Regione_TOSCANA,Regione_TRENTINOALTOADIGE,Regione_UMBRIA,Regione_VALLEDAOSTA,Regione_VENETO
2866,0.011938,0.010410,0.007369,0.007730,0.001499,0.002553,0.000539,-0.001151,0.000787,0.000876,...,0,0,0,0,0,0,0,0,0,0
6115,0.000702,0.000977,0.000319,0.000241,0.001166,0.000170,-0.000462,-0.000111,0.000405,0.000437,...,0,0,0,0,0,1,0,0,0,0
2182,0.008098,0.010086,0.000942,0.000929,0.000666,0.000681,0.000281,0.000676,0.000202,0.000180,...,0,0,0,0,0,0,0,0,0,0
4448,0.005215,0.005740,0.000960,0.001069,0.000666,0.000681,0.000216,0.000409,0.000349,0.000402,...,0,0,0,0,0,0,0,0,0,0
8500,0.005678,0.006952,0.001364,0.001640,0.002498,0.002553,0.000220,0.001479,0.000649,0.000723,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8772,0.011177,0.010166,0.000962,0.000786,0.000167,0.000000,0.000117,0.000126,0.000121,0.000119,...,0,0,0,0,0,0,0,0,0,1
163,0.009772,0.007222,0.002695,0.002190,0.003830,0.004595,0.000773,0.000853,0.000486,0.000674,...,0,0,0,0,0,0,0,0,0,1
6012,0.009655,0.010106,0.001644,0.001415,0.001332,0.001362,0.000138,0.000056,0.000252,0.000210,...,0,0,0,0,0,0,0,0,0,0
6558,0.009194,0.006992,0.006955,0.003826,0.002165,0.002042,0.009827,0.005389,0.002315,0.001958,...,0,0,0,0,0,1,0,0,0,0


In [27]:
validation

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,Regione_MOLISE,Regione_PIEMONTE,Regione_PUGLIA,Regione_SARDEGNA,Regione_SICILIA,Regione_TOSCANA,Regione_TRENTINOALTOADIGE,Regione_UMBRIA,Regione_VALLEDAOSTA,Regione_VENETO
8709,0.011449,0.011071,0.001386,0.001187,0.001499,0.001532,0.000630,0.000580,0.000215,0.000201,...,0,1,0,0,0,0,0,0,0,0
8790,0.006116,0.005395,0.000756,0.000696,0.001665,0.001702,0.000505,0.000444,0.000234,0.000245,...,0,0,0,0,0,1,0,0,0,0
1305,0.007897,0.007705,0.003884,0.003724,0.001998,0.002042,0.001377,0.001690,0.001067,0.001132,...,0,0,0,0,0,0,0,0,0,0
1328,0.001877,0.001674,0.001577,0.001157,0.000833,0.000000,0.003617,0.002508,0.006110,0.006329,...,0,0,0,0,0,0,0,0,0,0
3520,0.005755,0.005625,0.003113,0.002968,0.003331,0.003404,0.000673,0.000772,0.000937,0.000981,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5859,0.031956,0.034237,0.012346,0.010816,0.000833,0.001021,0.000423,0.000202,0.000431,0.000368,...,0,0,0,0,0,0,0,0,0,0
628,0.005722,0.007292,0.002818,0.002497,0.002165,0.002723,0.004514,0.004541,0.001705,0.001649,...,0,0,0,0,0,1,0,0,0,0
3693,0.012386,0.009883,0.005571,0.004770,0.001998,0.002042,0.002637,0.001706,0.000822,0.000895,...,0,0,0,0,0,0,1,0,0,0
5303,0.003011,0.003133,0.000062,0.000225,0.001499,0.001021,0.000073,-0.000207,0.000492,0.000339,...,0,0,0,0,0,0,0,0,0,0


# Bilanciamento
Abbiamo molte più osservazion in cui le aziende non sono fallite, quindi operiamo un bilanciamento tra i dati. Infatti, i dati sono passati da 11000 circa a 16000.

In [28]:
c_0 = train_df[train_df["Target"]==0] # Classe 0 individua tutti i casi che non sono malati, dentro il training
c_1 = train_df[train_df["Target"]==1] 
aumento = c_1.sample(len(c_0), replace = True)

In [29]:
train_df = pd.concat([c_0, aumento], axis = 0)
train_df

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,Regione_MOLISE,Regione_PIEMONTE,Regione_PUGLIA,Regione_SARDEGNA,Regione_SICILIA,Regione_TOSCANA,Regione_TRENTINOALTOADIGE,Regione_UMBRIA,Regione_VALLEDAOSTA,Regione_VENETO
2866,0.011938,0.010410,0.007369,0.007730,0.001499,0.002553,0.000539,-0.001151,0.000787,0.000876,...,0,0,0,0,0,0,0,0,0,0
6115,0.000702,0.000977,0.000319,0.000241,0.001166,0.000170,-0.000462,-0.000111,0.000405,0.000437,...,0,0,0,0,0,1,0,0,0,0
2182,0.008098,0.010086,0.000942,0.000929,0.000666,0.000681,0.000281,0.000676,0.000202,0.000180,...,0,0,0,0,0,0,0,0,0,0
4448,0.005215,0.005740,0.000960,0.001069,0.000666,0.000681,0.000216,0.000409,0.000349,0.000402,...,0,0,0,0,0,0,0,0,0,0
8500,0.005678,0.006952,0.001364,0.001640,0.002498,0.002553,0.000220,0.001479,0.000649,0.000723,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6844,0.007949,0.011916,0.006653,0.009814,0.003497,0.003574,-0.002607,0.001110,0.000804,0.001142,...,0,0,0,0,0,1,0,0,0,0
6138,0.000443,0.005151,0.002963,0.002898,0.002831,0.003744,-0.006262,0.001489,0.000979,0.001348,...,0,0,0,0,0,0,0,0,0,0
9361,0.005553,0.007643,0.002404,0.002679,0.003664,0.000000,-0.000263,0.001110,0.000454,0.000512,...,0,0,0,0,0,0,0,0,0,0
427,0.007606,0.006557,0.001148,0.001060,0.001166,0.001191,0.000060,0.000005,0.000248,0.000290,...,0,0,0,0,0,1,0,0,0,0


In [30]:
print(train_df["Target"].value_counts())

0    8154
1    8154
Name: Target, dtype: int64


In [31]:
# Droppiamo la variabile target e inseriamola in una variabile a sé

y_train = train_df["Target"]
X_train = train_df.drop(labels = ["Target"], axis = 1)
X_train

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,Regione_MOLISE,Regione_PIEMONTE,Regione_PUGLIA,Regione_SARDEGNA,Regione_SICILIA,Regione_TOSCANA,Regione_TRENTINOALTOADIGE,Regione_UMBRIA,Regione_VALLEDAOSTA,Regione_VENETO
2866,0.011938,0.010410,0.007369,0.007730,0.001499,0.002553,0.000539,-0.001151,0.000787,0.000876,...,0,0,0,0,0,0,0,0,0,0
6115,0.000702,0.000977,0.000319,0.000241,0.001166,0.000170,-0.000462,-0.000111,0.000405,0.000437,...,0,0,0,0,0,1,0,0,0,0
2182,0.008098,0.010086,0.000942,0.000929,0.000666,0.000681,0.000281,0.000676,0.000202,0.000180,...,0,0,0,0,0,0,0,0,0,0
4448,0.005215,0.005740,0.000960,0.001069,0.000666,0.000681,0.000216,0.000409,0.000349,0.000402,...,0,0,0,0,0,0,0,0,0,0
8500,0.005678,0.006952,0.001364,0.001640,0.002498,0.002553,0.000220,0.001479,0.000649,0.000723,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6844,0.007949,0.011916,0.006653,0.009814,0.003497,0.003574,-0.002607,0.001110,0.000804,0.001142,...,0,0,0,0,0,1,0,0,0,0
6138,0.000443,0.005151,0.002963,0.002898,0.002831,0.003744,-0.006262,0.001489,0.000979,0.001348,...,0,0,0,0,0,0,0,0,0,0
9361,0.005553,0.007643,0.002404,0.002679,0.003664,0.000000,-0.000263,0.001110,0.000454,0.000512,...,0,0,0,0,0,0,0,0,0,0
427,0.007606,0.006557,0.001148,0.001060,0.001166,0.001191,0.000060,0.000005,0.000248,0.000290,...,0,0,0,0,0,1,0,0,0,0


In [32]:
y_val = validation["Target"]
X_val = validation.drop(labels = ["Target"], axis = 1)
X_val

,Z1,Z2,Ricavi-1,Ricavi-2,Dipendenti-1,Dipendenti-2,EBITDA-1,EBITDA-2,Totale-1,Totale-2,...,Regione_MOLISE,Regione_PIEMONTE,Regione_PUGLIA,Regione_SARDEGNA,Regione_SICILIA,Regione_TOSCANA,Regione_TRENTINOALTOADIGE,Regione_UMBRIA,Regione_VALLEDAOSTA,Regione_VENETO
8709,0.011449,0.011071,0.001386,0.001187,0.001499,0.001532,0.000630,0.000580,0.000215,0.000201,...,0,1,0,0,0,0,0,0,0,0
8790,0.006116,0.005395,0.000756,0.000696,0.001665,0.001702,0.000505,0.000444,0.000234,0.000245,...,0,0,0,0,0,1,0,0,0,0
1305,0.007897,0.007705,0.003884,0.003724,0.001998,0.002042,0.001377,0.001690,0.001067,0.001132,...,0,0,0,0,0,0,0,0,0,0
1328,0.001877,0.001674,0.001577,0.001157,0.000833,0.000000,0.003617,0.002508,0.006110,0.006329,...,0,0,0,0,0,0,0,0,0,0
3520,0.005755,0.005625,0.003113,0.002968,0.003331,0.003404,0.000673,0.000772,0.000937,0.000981,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5859,0.031956,0.034237,0.012346,0.010816,0.000833,0.001021,0.000423,0.000202,0.000431,0.000368,...,0,0,0,0,0,0,0,0,0,0
628,0.005722,0.007292,0.002818,0.002497,0.002165,0.002723,0.004514,0.004541,0.001705,0.001649,...,0,0,0,0,0,1,0,0,0,0
3693,0.012386,0.009883,0.005571,0.004770,0.001998,0.002042,0.002637,0.001706,0.000822,0.000895,...,0,0,0,0,0,0,1,0,0,0
5303,0.003011,0.003133,0.000062,0.000225,0.001499,0.001021,0.000073,-0.000207,0.000492,0.000339,...,0,0,0,0,0,0,0,0,0,0


# Creazione Modello

Il modello scelto per effettuare la previsione è il Random Forest. Al fine di trovare i giusti parametri eseguiremo ora un for loop con una seriei di opzioni.

In [ ]:
list_n_estimators = []
list_max_features = []

lista_score = []
list_min_samples_split = []
list_min_samples_leaf = []

for n_estimators in range(200, 800, 100):
  for min_samples_split in range(3, 15, 1):
    for min_samples_leaf in range(3, 15, 1):
      for max_features in range(3, 15, 1):
        model = RandomForestClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features = max_features, random_state = 49) # Può analizzare massimo tre variabili per split
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        accuracy = accuracy_score(y_val, y_pred)
        list_n_estimators.append(n_estimators)
        lista_score.append(accuracy)
        list_min_samples_split.append(min_samples_split)
        list_min_samples_leaf.append(min_samples_leaf)
        list_max_features.append(max_features)
        print(f"N_estimators: {n_estimators}, Min Sample Split: {min_samples_split}, Min Sample Leaf: {min_samples_leaf}, Min Absolute Error {mae}, Accuracy: {accuracy}, Max Features: {max_features}")
        
max_value = max(lista_score)
max_index = lista_score.index(max_value)
print(f"Number of estimators = {list_n_estimators[max_index]}, Per min_samples_split = {list_min_samples_split[max_index]} e min_samples_leaf = {list_min_samples_leaf[max_index]}, number of features = {list_max_features[max_index]}, accuracy = {max_value} ")
    

In [34]:
clf = RandomForestClassifier(n_estimators=600, min_samples_split=13, min_samples_leaf=2, max_features = 3, random_state = 13) 

clf.fit(X_train, y_train)

RandomForestClassifier(max_features=3, min_samples_leaf=2, min_samples_split=13,
                       n_estimators=600, random_state=13)

In [35]:
y_pred = clf.predict(X_val) # Faccio una previsione

In [36]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2014
           1       0.84      0.71      0.77       186

    accuracy                           0.96      2200
   macro avg       0.90      0.85      0.87      2200
weighted avg       0.96      0.96      0.96      2200



In [37]:
confusion_matrix(y_val, y_pred)

array([[1988,   26],
       [  54,  132]])

In [38]:
y_prob = clf.predict_proba(X_val)
y_prob

# La probabilità che quella osservazione sia 0 e probabilità che sia 1 

array([[1.        , 0.        ],
       [0.99676471, 0.00323529],
       [0.97164868, 0.02835132],
       ...,
       [0.98749286, 0.01250714],
       [0.83412782, 0.16587218],
       [0.95727763, 0.04272237]])

# Sottomissione modello

Proviamo il modello sui nostri dati di test e prepariamolo per essere caricato su Kaggle.

In [39]:
y_test = clf.predict_proba(new_test)

In [40]:
new_df = pd.DataFrame(y_test)
new_df

,0,1
0,0.937562,0.062438
1,0.948420,0.051580
2,0.935560,0.064440
3,0.956131,0.043869
4,0.843214,0.156786
...,...,...
3961,0.929303,0.070697
3962,0.947260,0.052740
3963,0.970234,0.029766
3964,0.910980,0.089020


In [41]:
test["ID"]

0          1
1          2
2          3
3          4
4          5
        ... 
3961    3962
3962    3963
3963    3964
3964    3965
3965    3966
Name: ID, Length: 3966, dtype: int64

In [42]:
new_df["ID"] = test["ID"]

In [43]:
new_df2 = new_df.loc[:, new_df.columns != 0]
new_df2

,1,ID
0,0.062438,1
1,0.051580,2
2,0.064440,3
3,0.043869,4
4,0.156786,5
...,...,...
3961,0.070697,3962
3962,0.052740,3963
3963,0.029766,3964
3964,0.089020,3965


In [44]:
new_df2 = new_df2[["ID", 1]]
new_df2

,ID,1
0,1,0.062438
1,2,0.051580
2,3,0.064440
3,4,0.043869
4,5,0.156786
...,...,...
3961,3962,0.070697
3962,3963,0.052740
3963,3964,0.029766
3964,3965,0.089020


In [45]:
new_df2.rename(columns = {1:"Target"}, inplace = True)
new_df2

,ID,Target
0,1,0.062438
1,2,0.051580
2,3,0.064440
3,4,0.043869
4,5,0.156786
...,...,...
3961,3962,0.070697
3962,3963,0.052740
3963,3964,0.029766
3964,3965,0.089020


In [46]:
# Esportare il CSV
new_df2.to_csv(r'Random_Forest.csv', index = False, header = True)